In [1]:
import pandas as pd 
from numpy import int16

In [2]:
#DODAC ROZDIZAŁ NA ZLECENI Z PŁYTĄ PROSTOKĄTNĄ 1250x2500MM

In [3]:
#@title Funkcje
def generuj_tableke_na_cnc(_zlecenie):
  # zlecenie_ngl = _zlecenie.split("\n")[1].split("\t")[1:]
  zlecenie_ngl = ["MATERIAŁ",	"NUMER",	"ILOŚĆ",	"WYMIAR",	"OZNACZ.",	"CZAS"]
  zlecenie = list()
  for r in _zlecenie.split("\n")[2:]:
    r = r.split("\t")
    if len(r) == 7 and r[0] not in ["", "L.P."]:
      zlecenie.append(r[1:])

  df = pd.DataFrame(zlecenie, columns=zlecenie_ngl)

  df["GRUBOSC"] = df["MATERIAŁ"].apply(lambda x: x[-4:])


  def mat_typ(x):
    if "Sklejka" in x:
      return "SKL"
    elif "wiórowa" in x:
      return "PWI"

  def szer(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c not in "* ":
        new_str += c
    return new_str.strip().split("x")[0]

  def dlug(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c != "*" or c != " ":
        new_str += c
    return new_str.strip().split("x")[1]

  df["SZER"] = df["WYMIAR"].apply(szer).astype(int16)
  df["DLUG"] = df["WYMIAR"].apply(dlug).astype(int16)
  df["MAT_TYP"] = df["MATERIAŁ"].apply(mat_typ)
  df["ILOSC"] = df["ILOŚĆ"].astype(int16)

  return df

def drukuj_dane_cnc(df,MAT):

  _naglowek = "index$,material$,krok,sloje,grubosc,ilosc,mosty,nazwafmc$,kolor,fmcinclude$\n"
  content = list()
  content.append(_naglowek)
  # print(pwi_naglowek)
  for i in df[["NUMER", "ILOSC"]].values.tolist():
    if MAT == "S":
      a = [f"{i[0]}_SKL_X1_Msklejka,Sklejka,90,0,0,{i[1]},0,SKL",f"{i[0]}_SKL_X1_MSklejka.fmc,2,SKL","kontury"]
    elif MAT == "P":
      a = [f"{i[0]}_PWI_X1_MWIOR,WIOR,90,0,0,{i[1]},0,WIOR",f"{i[0]}_PWI_X1_MWIOR.fmc,2,WIOR","kontury"]
    b = chr(92).join(a)+chr(92)+"\n"
    # b = "/".join(a)+"/"+"\n"
    # print(b)
    content.append(b)

  content[-1] = content[-1].replace("\n", "")
  return content

def podziel_zlecenie(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df.groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:
      if f[0] == "PWI":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1])], "P")

      elif f[0] == "SKL":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1]) & (df.DLUG <= 1525)], "S")

      fi = open(f"{n}_{f[0]}_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

def podziel_zlecenie_SKLP(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df[(df.MAT_TYP=="SKL")].groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:

      content = drukuj_dane_cnc(df[(df.MAT_TYP=="SKL") & (df.GRUBOSC == f[1]) & (df.DLUG > 1525)], "S")

      fi = open(f"{n}_SKLP_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

In [4]:

#wzór na zlecenie
# """
# L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
# 1	Płyta wiórowa 16mm	1278	40	*** 115x245	CN	20
# 2	Płyta wiórowa 16mm	1279	80	*** 110x222	CN	40
# 3	Płyta wiórowa 16mm	1393	40	*** 158x260	CN	20
_zlecenie = {

"STO 4_1_24":
              """
L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
1	Płyta wiórowa 16mm	1605	120	*** 168x414	CN	2:00 (1)
2	Płyta wiórowa 16mm	1607	120	*** 150x398	CN	2:00 (1)
3	Płyta wiórowa 16mm	1771	60	*** 208x384	CN	1:00 (1)
4	Płyta wiórowa 16mm	1772	60	*** 190x368	CN	1:00 (1)
	RAZEM :		360			6:00



L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
1	Sklejka 18mm	1735	258	*** 140x512	CN	4:18 (1)
2	Sklejka 18mm	1736	466	*** 140x494	CN	7:46 (1)
3	Sklejka 18mm	1737	104	*** 122x494	CN	1:44 (1)
4	Sklejka 18mm	1738	40	*** 110x890	CN	0:40 (1)
5	Sklejka 18mm	1739	40	*** 110x990	CN	0:40 (1)
6	Sklejka 18mm	1740	20	*** 110x1454	CN	0:20 (1)
7	Sklejka 18mm	1742	332	*** 104x120	CN	5:32 (1)
8	Sklejka 18mm	1743	40	*** 470x470	CN	0:40 (1)
9	Sklejka 18mm	1744	40	*** 1240x1340	CN	0:40 (1)
10	Sklejka 18mm	1745	20	*** 1362x1462	CN	0:20 (1)
11	Sklejka 18mm	1746	20	*** 1380X1480	CN	0:20 (1)
12	Sklejka 18mm	1747	20	*** 1362x1462	CN	0:20 (1)
13	Sklejka 18mm	1748	20	*** 208x1129	CN	0:20 (1)
14	Sklejka 18mm	1749	20	*** 217x1235	CN	0:20 (1)
15	Sklejka 18mm	1752	20	*** 1462x1462	CN	0:20 (1)
16	Sklejka 18mm	1753	20	*** 1462x1462	CN	0:20 (1)
17	Sklejka 18mm	1754	40	*** 344x1098	CN	0:40 (1)
18	Sklejka 18mm	1755	40	*** 1340x1340	CN	0:40 (1)
19	Sklejka 18mm	1756	40	*** 600x600	CN	0:40 (1)
20	Sklejka 18mm	1757	40	*** 110x880	CN	0:40 (1)
21	Sklejka 18mm	1758	40	*** 110x860	CN	0:40 (1)
22	Sklejka 18mm	1759	40	*** 110x980	CN	0:40 (1)
23	Sklejka 18mm	1760	20	*** 110x1020	CN	0:20 (1)
24	Sklejka 18mm	1763	90	*** 110x828	CN	1:30 (1)
25	Sklejka 18mm	1764	120	*** 110x842	CN	2:00 (1)
26	Sklejka 18mm	1766	240	*** 110x842	CN	4:00 (1)
27	Sklejka 18mm	1770	40	*** 120x274	CN	0:40 (1)
28	Sklejka 18mm	1772	20	*** 1480x1480	CN	0:20 (1)
29	Sklejka 18mm	1781	15	*** 110x824	CN	0:15 (1)
30	Sklejka 18mm	1828	20	*** 110x872	CN	0:20 (1)
31	Sklejka 18mm	1829	10	*** 110x1012	CN	0:10 (1)
32	Sklejka 18mm	1830	30	*** 110x872	CN	0:30 (1)
33	Sklejka 18mm	1831	10	*** 110x250	CN	0:10 (1)
34	Sklejka 18mm	1931	110	*** 74x124	CN	1:50 (1)
35	Sklejka 18mm	1935	220	*** 74x104	CN	3:40 (1)
	RAZEM :		2665			44:25






 """

}


In [5]:
podziel_zlecenie(_zlecenie)

In [6]:
podziel_zlecenie_SKLP(_zlecenie)